In [5]:
# Import libraries

import numpy as np
from matplotlib import pyplot as plt


import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.functional import jacobian

In [10]:
def f(x):
    return lambda x: x[0] ** 2 + x[1] ** 2 + x[2] ** 2
def h1(x):
    return lambda x: ((x[0] ** 2) / 4) + ((x[1] ** 2) / 5) + ((x[2] ** 2) / 25) - 1
def h2(x):
    return lambda x: x[0] + x[1] - x[2]
x = Variable(torch.tensor([1.,1.,1.]), requires_grad=True)


We have $n=3$ and $m=2$ here, so the DOF of systme is $n-m = 1$ \
meanwe have only one decision variable variable and two state variable \
let say: \ 
$x_1 $ here is the decision variable \
$ x_2$  and  $x_3$ are the state variables \

In [30]:
# Calculating Reduce Gradient
Jocobian = torch.zeros((3, 3))
Jocobian[0] = jacobian(f(x), (x))
Jocobian[1] = jacobian(h1(x), (x))
Jocobian[2] = jacobian(h2(x), (x))

dfdd = Jocobian[0,0]
dfds = Jocobian[0,1:]
dhds = Jocobian[1:,1:]
dhdd = Jocobian[1:,0]

DfDd =  dfdd -torch.matmul(torch.matmul(dfds,torch.pinverse(dhds)),dhdd)
DfDd

tensor(-0.8333)